In [2]:
from sentence_transformers import SentenceTransformer
import csv
import torch

model_name = 'paraphrase-multilingual-mpnet-base-v2'
encoded_model_path = 'semantic_search_model.pt'
dataset_path = '/home/kkwon/AHN/paper_ft/cleaned_paper.csv'

bi_encoder = SentenceTransformer(model_name)

passages = []
with open(dataset_path) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        passages.append(row[0])
        
corpus_embeddings = bi_encoder.encode(
    passages, batch_size=32, convert_to_tensor=True, show_progress_bar=True)

torch.save(corpus_embeddings, encoded_model_path)

/home/kkwon/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/kkwon/.local/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/kkwon/.local/lib/python3.8/site-pack

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [1]:
import PyPDF2
import re
import pandas as pd

def pdf_to_text(pdf_path, skip_start_pages=0, skip_last_pages=0, header_lines=1, footer_lines=1):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        num_pages = len(pdf_reader.pages)
        
        print(f"Total pages in PDF: {num_pages}")

        # Adjust the range to process the correct pages
        start_page = skip_start_pages
        end_page = num_pages - skip_last_pages

        for page_num in range(start_page, end_page):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            
            if page_text:
                print(f"Page {page_num + 1}: {len(page_text)} characters extracted")
                lines = page_text.splitlines(True)[header_lines:-footer_lines]
                text += "".join(lines)
            else:
                print(f"Page {page_num + 1} is empty or could not be read")
        
        return text


# Define the input and output file paths
pdf_file_path = "/home/kkwon/AHN/paper_ft/datas/3362743.3362963.pdf"
output_file_path = "/home/kkwon/AHN/paper_ft/datas/paper1.txt"

# Extract text from the PDF file
# Adjust skip_start_pages and skip_last_pages as needed
raw_text = pdf_to_text(pdf_file_path, skip_start_pages=0, skip_last_pages=0, header_lines=2, footer_lines=1)

# Save the extracted text to a text file
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(raw_text)

print(f"Text extraction complete. Total characters extracted: {len(raw_text)}")

    
data=re.sub(r'[\n\t\r]',' ',raw_text)
sentences=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',data)
sentences=[sentence.strip() for sentence in sentences if sentence.strip()]
unique_sentences=list(dict.fromkeys(sentences))

df=pd.DataFrame(unique_sentences,columns=['Text'])
df.to_csv('/home/kkwon/AHN/paper_ft/cleaned_paper.csv',index=False)

Total pages in PDF: 6
Page 1: 6171 characters extracted
Page 2: 5578 characters extracted
Page 3: 4214 characters extracted
Page 4: 6023 characters extracted
Page 5: 2765 characters extracted
Page 6: 5628 characters extracted
Text extraction complete. Total characters extracted: 29507


In [5]:
import csv
from sentence_transformers import SentenceTransformer, util
import torch

bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
semantic_search_model = torch.load('semantic_search_model.pt')

passages = []
with open('/home/kkwon/AHN/paper_ft/cleaned_paper.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        passages.append(row[0])

question_embedding = bi_encoder.encode(
    "What hardware platforms were used for testing?", convert_to_tensor=True)
hits = util.semantic_search(
    question_embedding, semantic_search_model, top_k=3)
hits = hits[0]

result = {"search_results": [
    {"score": hit['score'], "text": passages[hit['corpus_id']]} for hit in hits]}

print(result)

{'search_results': [{'score': 0.5700194835662842, 'text': 'We test our implementation on two embedded devices which are representative of typical IoT devices you might see in use: Raspberry Pi Zero[ 5], Photon[ 9].'}, {'score': 0.516374945640564, 'text': 'Ideally, our hier- archical models would run on these native computing platforms.'}, {'score': 0.5103737115859985, 'text': 'However they evaluate their systems on relatively high powered devices which aren’t represen- tative of typical edge devices which have fewer resources.'}]}
